In [67]:
import pandas

frame = pandas.read_csv('gene_high_throughput_sequencing.csv').set_index('Patient_id')
frame.head()

,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
Patient_id,,,,,,,,,,,,,,,,,,,,,
STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,26.329928,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,39.140813,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,48.144685,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,27.076621,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,35.896701,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


# Часть 1: применение t-критерия Стьюдента

В первой части вам нужно будет применить критерий Стьюдента для проверки гипотезы о равенстве средних в двух независимых выборках. Применить критерий для каждого гена нужно будет дважды:

* для групп `normal` (control) и `early neoplasia` (treatment)
* для групп `early neoplasia` (control) и `cancer` (treatment)

В качестве ответа в этой части задания необходимо указать количество статистически значимых отличий, которые вы нашли с помощью t-критерия Стьюдента, то есть число генов, у которых p-value этого теста оказался меньше, чем уровень значимости.

In [68]:
normal_selector = frame['Diagnosis'] == 'normal'
early_neoplasia_selector = frame['Diagnosis'] == 'early neoplasia'
cancer_selector = frame['Diagnosis'] == 'cancer'
frame.drop('Diagnosis', axis=1, inplace=True)

In [106]:
%%time

from scipy import stats

test = pandas.DataFrame({'Group': ['normal vs early neoplasia', 'early neoplasia vs cancer']}).set_index('Group')
for column in frame.columns:
    test[column] = [
        stats.ttest_ind(frame[column][normal_selector], frame[column][early_neoplasia_selector], equal_var=False).pvalue,
        stats.ttest_ind(frame[column][early_neoplasia_selector], frame[column][cancer_selector], equal_var=False).pvalue,
    ]

CPU times: user 49.9 s, sys: 1.65 s, total: 51.6 s
Wall time: 51.7 s


In [107]:
test

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
Group,,,,,,,,,,,,,,,,,,,,,
normal vs early neoplasia,0.690766,0.000032,0.060273,0.826429,0.049876,0.144267,0.329108,0.023712,0.240148,0.037933,...,0.463274,0.806008,0.424543,0.740591,0.414922,0.640904,0.830134,0.670395,0.793925,0.661031
early neoplasia vs cancer,0.413735,0.653429,0.079556,0.287581,0.463292,0.007681,0.481306,0.578830,0.000740,0.712687,...,0.107366,0.458907,0.893433,0.467608,0.881584,0.659369,0.330617,0.542939,0.565753,0.639010


In [108]:
def print_answer(frame, name):
    assert frame.shape == (1,), frame.shape
    file=open(name, 'wt').write(str(frame.values[0]))

answer_1 = pandas.DataFrame((test < 0.05).sum(axis=1))
print_answer(answer_1.loc['normal vs early neoplasia'], 'diffierientsial-no-ekspriessirovannyie-ghieny-1.1.txt')
print_answer(answer_1.loc['early neoplasia vs cancer'], 'diffierientsial-no-ekspriessirovannyie-ghieny-1.2.txt')
answer_1

,0
Group,
normal vs early neoplasia,1575
early neoplasia vs cancer,3490


# Часть 2: поправка методом Холма

В этой части задания нужно будет применить поправку Холма для получившихся двух наборов достигаемых уровней значимости из предыдущей части. Обратите внимание, что поскольку вы будете делать поправку для каждого из двух наборов p-value отдельно, то проблема, связанная с множественной проверкой останется.

Для того, чтобы ее устранить, достаточно воспользоваться поправкой Бонферрони, то есть использовать уровень значимости `0.05 / 2` вместо `0.05` для дальнейшего уточнения значений p-value c помощью метода Холма.

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Холма-Бонферрони. Причем это число нужно ввести с учетом практической значимости: посчитайте для каждого значимого изменения fold change и выпишите в ответ число таких значимых изменений, абсолютное значение fold change которых больше, чем `1.5`.

Обратите внимание, что

* применять поправку на множественную проверку нужно **ко всем значениям достигаемых уровней значимости, а не только для тех, которые меньше значения уровня доверия**.
* при использовании поправки на уровне значимости `0.025` **меняются значения достигаемого уровня значимости, но не меняется значение уровня доверия** (то есть для отбора значимых изменений скорректированные значения уровня значимости нужно сравнивать с порогом `0.025`, а не `0.05`)!

In [109]:
%%time

def get_fold_change(control: float, treatment: float) -> float:
    return treatment / control if treatment > control else control / treatment

fold_change = pandas.DataFrame({'Group': ['normal vs early neoplasia', 'early neoplasia vs cancer']}).set_index('Group')
for column in frame.columns:
    fold_change[column] = [
        get_fold_change(frame[column][normal_selector].mean(), frame[column][early_neoplasia_selector].mean()),
        get_fold_change(frame[column][early_neoplasia_selector].mean(), frame[column][cancer_selector].mean()),
    ]

CPU times: user 40.9 s, sys: 1.51 s, total: 42.5 s
Wall time: 42.5 s


In [110]:
fold_change

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
Group,,,,,,,,,,,,,,,,,,,,,
normal vs early neoplasia,1.067858,1.996517,1.222418,1.020070,1.125471,1.151327,1.111462,1.113459,1.066094,1.204175,...,1.091228,1.075545,1.248388,1.116410,1.176629,1.152672,1.026978,1.117197,1.097481,1.101721
early neoplasia vs cancer,1.148593,1.064871,1.278517,1.092635,1.036574,1.285160,1.074193,1.038536,2.458867,1.032005,...,1.221304,1.297719,1.044923,1.297491,1.043030,1.168987,1.115262,1.225243,1.290314,1.149064


In [111]:
from statsmodels.stats.multitest import multipletests

test_holm = test.apply(lambda row: multipletests(row, method='holm', alpha=0.025)[0], axis=1)
test_holm

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
Group,,,,,,,,,,,,,,,,,,,,,
normal vs early neoplasia,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
early neoplasia vs cancer,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [112]:
def combine(fold_change, test):
    return pandas.DataFrame(((fold_change > 1.5) & test).sum(axis=1))
    
answer_2 = combine(fold_change, test_holm)
print_answer(answer_2.loc['normal vs early neoplasia'], 'diffierientsial-no-ekspriessirovannyie-ghieny-2.1.txt')
print_answer(answer_2.loc['early neoplasia vs cancer'], 'diffierientsial-no-ekspriessirovannyie-ghieny-2.2.txt')
answer_2

,0
Group,
normal vs early neoplasia,2
early neoplasia vs cancer,77


# Часть 3: поправка методом Бенджамини-Хохберга

Данная часть задания аналогична второй части за исключением того, что нужно будет использовать метод Бенджамини-Хохберга.

Обратите внимание, что методы коррекции, которые контролируют FDR, допускает больше ошибок первого рода и имеют большую мощность, чем методы, контролирующие FWER. Большая мощность означает, что эти методы будут совершать меньше ошибок второго рода (то есть будут лучше улавливать отклонения от H0, когда они есть, и будут чаще отклонять H0, когда отличий нет).

В качестве ответа к этому заданию требуется ввести количество значимых отличий в каждой группе после того, как произведена коррекция Бенджамини-Хохберга, причем так же, как и во второй части, считать только такие отличия, у которых `abs(fold change) > 1.5`.

In [113]:
test_fdr = test.apply(lambda row: multipletests(row, method='fdr_bh', alpha=0.025)[0], axis=1)
test_fdr

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
Group,,,,,,,,,,,,,,,,,,,,,
normal vs early neoplasia,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
early neoplasia vs cancer,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [114]:
answer_3 = combine(fold_change, test_fdr)
print_answer(answer_3.loc['normal vs early neoplasia'], 'diffierientsial-no-ekspriessirovannyie-ghieny-3.1.txt')
print_answer(answer_3.loc['early neoplasia vs cancer'], 'diffierientsial-no-ekspriessirovannyie-ghieny-3.2.txt')
answer_3

,0
Group,
normal vs early neoplasia,4
early neoplasia vs cancer,524
